# [MATH2504 Programming of Simulation, Analysis, and Learning Systems at The University of Queensland](https://courses.smp.uq.edu.au/MATH2504/)

## Semester 2, 2021

# Practical D: Practice for Quiz

In this practical we prepare for the quiz.

**Task 1:** See the [practice quiz](https://courses.smp.uq.edu.au/MATH2504/quiz/practice_quiz_2021.pdf)

**Sub-Task:** Make sure you read the instructions on the first page. The same instructions will be used for the first quiz.

**Sub-Task:** Consider question 1. The solutions are:

In [1]:
s = sum([i^2 for i in 1:100])
println(s)

338350


In [3]:
s = 0
for i in 1:100  #Solution of Q1a
    s += i^2    #Solution of Q1a
end
println(s)

338350


In [6]:
s, i = 0, 1
while true
    s += i^2
    i == 100 ? break : i += 1 #Solution of Q1b
end
println(s)

338350


In [9]:
function f(s,i)
    if i == 0
        return s, 0
    else
        return f(s + i^2, i-1)  #Solution of Q1c
    end
end
s, _ = f(0,100)
println(s)

338350


**Sub-Task:** Consider question 2. 

In [10]:
function to_value(x::UInt8)
    n = x >> 3
    R = x & 0b111
    @assert R<=5
    (n == 0 ? 0 : UInt128(10)^n) + R
end

to_value (generic function with 1 method)

The solutions are:

**Solution for Q2a:** `0xFF` is a byte with all bits set to 1. This means that `n=31` and `R=7`. Hence we will get an assertion error.

In [11]:
to_value(0xFF)

LoadError: AssertionError: R <= 5

**Solution for Q2b:** The largest value without an error would have bits `11111qqq` where `qqq` are set to a value of decimal 5, or `qqq = 101`. That is the largest value is `11111101 = 0xFD` because `1101` in binary is `D` in hex.

In [16]:
to_value(0xFD)

0x0000007e37be2022c0914b2680000005

In [18]:
#Try to_value(0xFE) #(too large)

**Solution for Q2c:** 
The line `R = x & 0b111` zeroes out the 5 top bits of `x` and puts the remaining values in `R`. This can also be done via, `R = (x << 5) >> 5`.

In [25]:
function to_value2(x::UInt8)
    n = x >> 3
    R = (x << 5) >> 5
    @assert R<=5
    (n == 0 ? 0 : UInt128(10)^n) + R
end

to_value2 (generic function with 1 method)

In [26]:
to_value(0xFD), to_value2(0xFD)

(0x0000007e37be2022c0914b2680000005, 0x0000007e37be2022c0914b2680000005)

**Sub-Task:** Consider question 3.

Quiz question 3a:

In [35]:
function is_sort(a)
    n = length(a)
    n == 1 && return true #The ??? should be true (solution to 3a)
    m = n ÷ 2
    return is_sort(a[1:m]) && is_sort(a[(m+1):n]) && a[m] <= a[m+1]
end

is_sort (generic function with 1 method)

In [36]:
is_sort([1,2,3])

true

In [37]:
is_sort([1,3,2])

false

Quiz question 3b:

In [39]:
is_sort([1,2,4,3,10,7])

false

On first call (**1**) we have `n=6` and hence `m=3`. We then call the function on `[1,2,4]` (call **2**) and again on `[3,10,7]` (call **3**).

On call to `[1,2,4]` we have `n=3` and hence `m=1`. We then call the function on `[1]` (call **4**), and `[2,4]` (call **5**).

On call to `[1]` we return `true`. Then on call to `[2,4]` we have `n=2` and hence `m=1`. We then call on `[2]` (call **6**) and again on `[4]` (call **7**). Each of these returns true.

Now the call to `[3,10,7]` operates similarly to the call on `[1,2,4]`. It has calls **8**, **9**, **10**, and **11**.

Hence in total there are 11 calls in total.

In [46]:
#Here is a profiling demonstration of this
call_counter = 0
function is_sort_profile(a)
    global call_counter += 1
    @show call_counter
    n = length(a)
    n == 1 && return true #The ??? should be true (solution to 3a)
    m = n ÷ 2
    return is_sort_profile(a[1:m]) && is_sort_profile(a[(m+1):n]) && a[m] <= a[m+1]
end
is_sort_profile([1,2,4,3,10,7])

call_counter = 1
call_counter = 2
call_counter = 3
call_counter = 4
call_counter = 5
call_counter = 6
call_counter = 7
call_counter = 8
call_counter = 9
call_counter = 10
call_counter = 11


false

Quiz question 3c:

If we Switch the order of evaluation in the last line of the function then in certain cases we won't need to sort sublists due to short-circuting and evaluation. Here we demonstrate it on the profiling version:

In [51]:
#Here is a profiling demonstration of this
call_counter = 0
function is_sort_profile2(a)
    global call_counter += 1
    @show call_counter
    n = length(a)
    n == 1 && return true #The ??? should be true (solution to 3a)
    m = n ÷ 2
    return a[m] <= a[m+1] && is_sort_profile2(a[1:m]) && is_sort_profile2(a[(m+1):n])
end;

In [52]:
call_counter = 0
is_sort_profile([1,2,4,3,10,7])

call_counter = 1


false

## Task 2: Additional short questions

**2a:** Consider this code taken from [Unit 1](https://courses.smp.uq.edu.au/MATH2504/lectures_html/lecture-unit-1.html):

In [61]:
half_adder(a::T, b::T) where {T <: Bool} = (xor(a,b), a & b)
 
function full_adder(a::T, b::T, carry::T) where {T<:Bool}
    (s, carry_a) = half_adder(carry, a)
    (s, carry_b) = half_adder(s, b)
    return (s, carry_a | carry_b)
end

function add_n_digits(a::T, b::T, n) where {T <: Int}
    #calculate y = a + b using array indexing and logic operations
    #convert to a vector of booleans
    a_bool = digits(Bool, a, base = 2, pad = n)
    b_bool = digits(Bool, b, base = 2, pad = n)
    y_bool = fill(Bool(0), n)
    carry = false
    for i = 1 : n
        (y_bool[i], carry) = full_adder(a_bool[i], b_bool[i], carry)
    end
    return y_bool, a_bool, b_bool
end

#turn boolean array back into a string
display_bool_array(y) = join(reverse(Int.(y)), "")

a = 1
b = 1

y_bool, a_bool, b_bool = add_n_digits(a, b, 16)

println("adding $a + $b")
println("a = $(display_bool_array(a_bool))")
println("b = $(display_bool_array(b_bool))")
println("y = $(display_bool_array(y_bool))", "\n")

a = 1
b = 2^16 - 1 #biggest 16 bit UInt

y_bool, a_bool, b_bool = add_n_digits(a, b, 16)

println("adding $a + $b")
println("a = $(display_bool_array(a_bool))")
println("b = $(display_bool_array(b_bool))")
println("y = $(display_bool_array(y_bool))", "\n")
a

1

Implement code similar to it for subtraction.

### Finding a value in a sorted array

**2b**: Assume you have numbers in a sorted array with $n$ entries. You then wish to implement a function which will check if a given number is in the array.

**2b-i:** What is the time complexity of this operation if you search through the entries sequentially, starting at index $1$, moving onto index $2$, and until index $n$.

The operation can be imporved by searching via a bisection search: 

In [7]:
function in_sorted_array(a::Vector{T}, x::T) where T<:Real
    n = length(a)
    n == 0 && return false
    m = n ÷ 2 + 1
    x == a[m] && return true
    x < a[m] && return in_sorted_array(a[1:m-1], x)
    return in_sorted_array(a[(m+1):end], x)
end

in_sorted_array (generic function with 1 method)

In [14]:
#Example
using Random; Random.seed!(0)
data = rand(Int,20)
data[17] = 1001
sort!(data)

in_sorted_array(data,1001)

true

**2b-ii:** Determine the complexity of this sorting operation and argue your result. Ignore the fact that the current implemintation above copies the arrays on each recursive call.

**2b-iii:** Implement the same algorithm without using recursion. 

**2c:** Floating point numbers

Assume a hypothetical floating point number with $d=3$ bits for the significant, $2$ bits for the exponent, and a sign bit. Determine the range of values that can be represented via this 6 bit floating point number.